In [2]:
import requests
import json
import csv
import pandas as pd
import time
import re
from pathlib import Path

def print_r(text):
    print(json.dumps(text, indent=4, sort_keys=False))


In [3]:
api_key = "4tsbjxvbrwnpcjdsh277csqu"
url = "http://api.walmartlabs.com/v1/search" 
params = {
    "apiKey": api_key,
    "format": "json", # json|xml
    "numItems": 25,
    "sort": "bestseller", # relevance, price, title, bestseller, customerRating, new
    "order": "desc", # asc, desc
    "facet":"on",
    "facet.range":"price:[151 TO 200]",
    #"start": 1,
    #"categoryId": "3944_3951_1089430_1230093_1230101"
}
params["query"] = "laptop"

In [4]:
def walmart_search(q):
    params["query"] = q
    walmart_data = requests.get(url, params=params)
    walmart_url = walmart_data.url
    try:
        walmart_data = walmart_data.json()
    except ValueError:
        print("Decoding JSON has failed in:")
        print(walmart_url)
        walmart_data = {
            "DecodeERROR":1
        }
    #print(walmart_url)
    return walmart_data


In [21]:
#print_r(params)
params["start"] = 1


query = "Laptop"
startpage = 0
maxpages = 180
defer = 50

query = "TV"
startpage = 0
maxpages = 80
defer = 100

query = "Phone"
startpage = 2
maxpages = 80
defer = 50

#out_file = open("walmart_"+query+"_data_file3.csv","w")
out_file = "Walmart_datasets/walmart_"+query+"_data_file.csv"
all_data =[]

for xx in range(startpage,maxpages):
    strt_price = xx*defer+1
    end_price = xx*defer+defer
    out_file2 = "output/walmart_"+query+"_"+str(strt_price)+"-"+str(end_price)+"_data_file.csv"
    my_file = Path(out_file2)
    if my_file.is_file():
        # file exists
        #print("file exists")
        read_df = pd.read_csv(out_file2)
        #read_df.head()
        for index, row in read_df.iterrows():
            item_data = {}
            item_data["itemId"]            = row["itemId"]
            item_data["parentItemId"]      = row["parentItemId"]
            item_data["name"]              = row["name"]
            item_data["salePrice"]         = row["salePrice"]
            item_data["upc"]               = row["upc"]
            item_data["categoryPath"]      = row["categoryPath"]
            item_data["numReviews"]        = row["numReviews"]
            item_data["customerRating"]    = row["customerRating"]
            item_data["thumbnailImage"]    = row["thumbnailImage"]
            item_data["mediumImage"]       = row["mediumImage"]
            item_data["largeImage"]        = row["largeImage"]
            item_data["categoryNode"]      = row["categoryNode"]
            item_data["offerType"]         = row["offerType"]
            m = re.findall(r'[0-9\.]+"', row["name"])
            #item_data["screensize"]    = re.findall(r'[0-9\.]+"', row["name"])[0].replace("\"","")
            if len(m) >0:
                m = m[0].replace("\"","")
            else:
                m = 0
            #print(m)
            item_data["screensize"]    = m
            all_data.append(item_data)
    else:
        prices = "price:["+str(strt_price)+" TO "+str(end_price)+"]"
        all_data2 =[]
        params["facet.range"] = prices
        #print(params)
        ret = walmart_search(query)
        if "totalResults" not in ret:
            continue
        totalResults =ret["totalResults"]
        if totalResults==0:
            continue
        pages2 = totalResults/25+1
        pages = int(round(pages2,0))
        if pages2>pages:
            pages +=1
        #print(pages)
        print("Looking for ("+query+") price range ("+str(strt_price)+","+str(end_price)+"), Total results ("+str(totalResults)+") in ("+str(pages-1)+") Pages")
        for i in range(1,pages):
            time.sleep(1)
            strt = (i*25) - 24;
            if strt<=1000:
                params["start"] = strt
                try:
                    if strt!=1:
                        ret = walmart_search(query)
                    if "DecodeERROR" not in ret:
                        if ret["numItems"]>0:
                            #print_r(ret)
                            #print(ret["items"][0]["name"])
                            for x in ret["items"]:
                                try:
                                    if "numReviews" not in x:
                                        x["numReviews"]=0
                                        x["customerRating"]="0.0"
                                    if "largeImage" not in x:
                                        x["largeImage"]="0.0"
                                    if "salePrice" in x and "upc" in x:
                                        if "\\" not in x["name"]:
                                            item_data = {}
                                            item_data["itemId"] = x["itemId"]
                                            item_data["parentItemId"] = x["parentItemId"]
                                            try:
                                            #    out_body.append(str(x["name"]))
                                                item_data["name"] = x["name"]
                                            except (KeyError, IndexError):
                                            #    out_body.append(str("Error with the name"))
                                                item_data["name"] = "Error with the name"

                                            item_data["salePrice"] = x["salePrice"]
                                            item_data["upc"] = x["upc"]
                                            item_data["categoryPath"] = x["categoryPath"]
                                            item_data["numReviews"] = x["numReviews"]
                                            item_data["customerRating"] = x["customerRating"]
                                            item_data["thumbnailImage"] = x["thumbnailImage"]
                                            item_data["mediumImage"] = x["mediumImage"]
                                            item_data["largeImage"] = x["largeImage"]
                                            item_data["categoryNode"] = x["categoryNode"]
                                            item_data["offerType"] = x["offerType"]
                                            m = re.findall(r'[0-9\.]+"', x["name"])
                                            #item_data["screensize"]    = re.findall(r'[0-9\.]+"', row["name"])[0].replace("\"","")
                                            if len(m) >0:
                                                m = m[0].replace("\"","")
                                            else:
                                                m = 0
                                            #print(m)
                                            item_data["screensize"]    = m
                                            all_data.append(item_data)
                                            all_data2.append(item_data)
                                except (KeyError, IndexError):
                                    print("Missing field/result... skipping.")

                except (KeyError, IndexError):
                    print("Missing field/result... skipping.")
            #print(row["upc"], index)
        results_pd2 = pd.DataFrame(all_data2)
        results_pd2.to_csv(out_file2, sep=',')

print("----------------------------------")
print("We Finished, but not yet")


----------------------------------
We Finished, but not yet


In [22]:
#print(all_data)
results_pd = pd.DataFrame(all_data)
results_pd.to_csv(out_file, sep=',')
results_pd


,categoryNode,categoryPath,customerRating,itemId,largeImage,mediumImage,name,numReviews,offerType,parentItemId,salePrice,screensize,thumbnailImage,upc
0,1085632_1094344_2365074,Seasonal/Father's Day/Father's Day Electronics,0.0,712004929,https://i5.walmartimages.com/asr/f9ddc826-fa3d...,https://i5.walmartimages.com/asr/f9ddc826-fa3d...,ZOJI-Z7 4G Mobile Cell Phone Quad Core Fingerp...,0,ONLINE_ONLY,712004929,127.64,0,https://i5.walmartimages.com/asr/f9ddc826-fa3d...,192278851443
1,1229749_1070964_3977,Office/Office Technology/Phones & Accessories/...,0.0,774166579,https://i5.walmartimages.com/asr/44d9a4d9-7b01...,https://i5.walmartimages.com/asr/44d9a4d9-7b01...,"""Vtech CS6909 (4-Pack) Accessory Handset for C...",0,ONLINE_ONLY,774166579,104.85,0,https://i5.walmartimages.com/asr/44d9a4d9-7b01...,657379498404
2,1229749_1070964_3977,Office/Office Technology/Phones & Accessories/...,0.0,138382833,https://i5.walmartimages.com/asr/289b3510-23a1...,https://i5.walmartimages.com/asr/289b3510-23a1...,Escence 2 SIP Account IP Phone,0,ONLINE_ONLY,138382833,149.00,0,https://i5.walmartimages.com/asr/289b3510-23a1...,700443222761
3,1229749_1070964_3977,Office/Office Technology/Phones & Accessories/...,0.0,519023948,https://i5.walmartimages.com/asr/087c8a72-2c70...,https://i5.walmartimages.com/asr/087c8a72-2c70...,IP PHONE 7841 FOR 3RD PARTY CALL CONTROL,0,ONLINE_ONLY,519023948,140.70,0,https://i5.walmartimages.com/asr/087c8a72-2c70...,882658829802
4,1105910_1072335_1231295,Cell Phones/No-Contract Phones & Plans/Shop Ph...,0.0,460217854,https://i5.walmartimages.com/asr/f1488e6d-5a56...,https://i5.walmartimages.com/asr/f1488e6d-5a56...,Indigi® 5.6inch 2018 4G LTE Unlocked Android 6...,0,ONLINE_ONLY,460217854,129.99,0,https://i5.walmartimages.com/asr/f1488e6d-5a56...,798711873555
5,1229749_1070964_3977,Office/Office Technology/Phones & Accessories/...,0.0,38595260,https://i5.walmartimages.com/asr/a0b3760c-17df...,https://i5.walmartimages.com/asr/a0b3760c-17df...,"""AT&T CL80101 (5-Pack) Cordless Extension Hand...",0,ONLINE_ONLY,38595260,112.92,0,https://i5.walmartimages.com/asr/a0b3760c-17df...,616174199923
6,1105910_1231203_1231205,Cell Phones/Shop Phones by Operating System/An...,0.0,589207749,https://i5.walmartimages.com/asr/a94fdc49-63df...,https://i5.walmartimages.com/asr/a94fdc49-63df...,Portable R11s 5.5 Inch 3G QHD Quad-core Mobile...,0,ONLINE_ONLY,589207749,109.40,0,https://i5.walmartimages.com/asr/a94fdc49-63df...,192278850620
7,1105910_1231203_1231205,Cell Phones/Shop Phones by Operating System/An...,0.0,948442775,https://i5.walmartimages.com/asr/2c9f0ea3-1562...,https://i5.walmartimages.com/asr/2c9f0ea3-1562...,"OUKITEL K4000 Plus 5"" HD Screen 4100mAh Press ...",0,ONLINE_ONLY,948442775,133.55,5,https://i5.walmartimages.com/asr/2c9f0ea3-1562...,192278851542
8,1105910_1231203_1231205,Cell Phones/Shop Phones by Operating System/An...,0.0,429970579,https://i5.walmartimages.com/asr/f50d7599-abe3...,https://i5.walmartimages.com/asr/f50d7599-abe3...,"CUBOT X16 5.0"" 4G LTE FHD Android 5.1 MT6735 Q...",0,ONLINE_ONLY,429970579,142.86,5.0,https://i5.walmartimages.com/asr/f50d7599-abe3...,661909435050
9,1229749_1070964_3977,Office/Office Technology/Phones & Accessories/...,0.0,33279029,https://i5.walmartimages.com/asr/2e1aa35d-1e82...,https://i5.walmartimages.com/asr/2e1aa35d-1e82...,Serene Innovations HD talking CID cordless,0,ONLINE_ONLY,33279029,111.89,0,https://i5.walmartimages.com/asr/2e1aa35d-1e82...,804879180500


In [23]:
bestbuy_api_key = "Jz4AJitfYbGIGGaOuTR09Lpl"
bestbuyUrl = "https://api.bestbuy.com/v1/products"

show_arr=[
    #"accessories.sku",
    #"color",
    "condition",
    "customerReviewAverage",
    "customerReviewCount",
    "customerTopRated",
    #"depth",
    "description",
    "department",
    "departmentId",
    #"details.name",
    #"details.value",
    "digital",
    #"features.feature",
    "format",
    "height",
    "salePrice",
    #"includedItemList.includedItem",
    "longDescription",
    #"longDescriptionHtml",
    "manufacturer",
    "modelNumber",
    "name",
    "preowned",
    #"productVariations.sku",
    "quantityLimit",
    "releaseDate",
    "shortDescription",
    "shortDescriptionHtml",
    "sku",
    "upc",
    "warrantyLabor",
    "inStoreAvailability",
    "onlineAvailability",
    "warrantyParts",
    "weight",
    "width"
    ,"image"
    ,"largeImage"
    ,"thumbnailImage"
]
params2 = {
    "apiKey": bestbuy_api_key,
    "page": 1,
    "show": ",".join(show_arr),
    "pageSize": 100,
    "format": "json" # json|xml
}
params2["query"] = "ipad"
print(bestbuyUrl)


https://api.bestbuy.com/v1/products


In [24]:
def bestbuy_search(q):
    urlquery = bestbuyUrl + "(search=" + q + ")"
    bestbuy_data = requests.get(urlquery, params=params2)
    bestbuy_url = bestbuy_data.url
    bestbuy_data = bestbuy_data.json()
    #print_r(bestbuy_url)
    return bestbuy_data

In [31]:
query = "TV"
query = "Phone"
query = "Laptop"



out_file = "Walmart_datasets/bestbuy_"+query+"_data_file.csv"
all_data=[]
ret = bestbuy_search(query)
if "total" in ret:
    #print_r(ret)
    total = ret["total"]
    totalPages = ret["totalPages"]
    prices = "price:["+str(strt_price)+" TO "+str(end_price)+"]"
    all_data2 =[]
    for xx in range(1,(totalPages+1)):
    #for xx in range(1,(1+1)):
        page = xx
        out_file2 = "output/bestbuy_"+query+"_"+str(page)+"_data_file.csv"
        my_file = Path(out_file2)
        if my_file.is_file():
            # file exists
            #print("file exists")
            read_df = pd.read_csv(out_file2)
            #read_df.head()
            for index, row in read_df.iterrows():
                item_data = {}
                item_data["itemId"]         = row["itemId"]
                item_data["department"]   = row["department"]
                item_data["name"]           = row["name"]
                item_data["salePrice"]      = row["salePrice"]
                item_data["upc"]            = row["upc"]
                item_data["departmentId"]   = row["departmentId"]
                item_data["numReviews"]     = row["numReviews"]
                item_data["customerRating"] = row["customerRating"]
                item_data["thumbnailImage"] = row["thumbnailImage"]
                item_data["mediumImage"]    = row["mediumImage"]
                item_data["largeImage"]     = row["largeImage"]
                #item_data["categoryNode"]   = row["categoryNode"]
                item_data["inStoreAvailability"]      = row["inStoreAvailability"]
                item_data["onlineAvailability"]      = row["onlineAvailability"]
                item_data["manufacturer"]      = row["manufacturer"]
                item_data["quantityLimit"]      = row["quantityLimit"]
                item_data["manufacturer"]      = row["manufacturer"]
                m = re.findall(r'[0-9\.]+"', row["name"])
                #item_data["screensize"]    = re.findall(r'[0-9\.]+"', row["name"])[0].replace("\"","")
                if len(m) >0:
                    m = m[0].replace("\"","")
                else:
                    m = 0
                #print(m)
                item_data["screensize"]    = m
                all_data.append(item_data)
        else:
            all_data2 =[]
            params2["page"] = page
            #print(params2)
            time.sleep(1)
            if(page>1):
                ret = bestbuy_search(query)
            print("Looking for ("+query+") page ("+str(page)+"), Total results ("+str(total)+") in ("+str(totalPages)+") Pages")
            #try:
            if total>0:
                #print_r(ret)
                #print(ret["items"][0]["name"])
                #print_r(ret)
                for x in ret["products"]:
                    #try:
                    item_data = {}
                    item_data["itemId"]         = x["sku"]
                    item_data["department"]   = x["department"]
                    item_data["name"]           = x["name"]
                    item_data["salePrice"]      = x["salePrice"]
                    item_data["upc"]            = x["upc"]
                    item_data["departmentId"]   = x["departmentId"]
                    item_data["numReviews"]     = x["customerReviewCount"]
                    item_data["customerRating"] = x["customerReviewAverage"]
                    item_data["thumbnailImage"] = x["thumbnailImage"]
                    item_data["mediumImage"]    = x["image"]
                    item_data["largeImage"]     = x["largeImage"]
                    #item_data["categoryNode"]   = x["categoryNode"]
                    item_data["inStoreAvailability"]      = x["inStoreAvailability"]
                    item_data["onlineAvailability"]      = x["onlineAvailability"]
                    item_data["manufacturer"]      = x["manufacturer"]
                    item_data["quantityLimit"]      = x["quantityLimit"]
                    item_data["manufacturer"]      = x["manufacturer"]
                    m = re.findall(r'[0-9\.]+"', x["name"])
                    if len(m) >0:
                        m = m[0].replace("\"","")
                    else:
                        m = 0
                    #print(m)
                    item_data["screensize"]    = m
                    #item_data["screensize"]    = re.findall(r'[0-9\.]+"', x["name"])[0].replace("\"","")
                    all_data.append(item_data)
                    all_data2.append(item_data)
                    #except (KeyError, IndexError):
                    #    print("Missing field/result... skipping.")

            #except (KeyError, IndexError):
                #print("Missing field/result... skipping.")
                #print(row["upc"], index)
            results_pd2 = pd.DataFrame(all_data2)
            results_pd2.to_csv(out_file2, sep=',')

print("----------------------------------")
print("We Finished, but not yet")


----------------------------------
We Finished, but not yet


In [32]:
results_pd = pd.DataFrame(all_data)
results_pd.to_csv(out_file, sep=',')
results_pd

read_df = pd.read_csv(out_file)
all_data2=[]
for index, row in read_df.iterrows():
    #print_r(row)
    item_data = {}
    item_data["itemId"]         = row["itemId"]
    item_data["department"]   = row["department"]
    item_data["name"]           = row["name"]
    item_data["salePrice"]      = row["salePrice"]
    item_data["upc"]            = row["upc"]
    item_data["departmentId"]   = row["departmentId"]
    item_data["numReviews"]     = row["numReviews"]
    item_data["customerRating"] = row["customerRating"]
    item_data["thumbnailImage"] = row["thumbnailImage"]
    item_data["mediumImage"]    = row["mediumImage"]
    item_data["largeImage"]     = row["largeImage"]
    #item_data["categoryNode"]   = row["categoryNode"]
    item_data["inStoreAvailability"]      = row["inStoreAvailability"]
    item_data["onlineAvailability"]      = row["onlineAvailability"]
    item_data["manufacturer"]      = row["manufacturer"]
    item_data["quantityLimit"]      = row["quantityLimit"]
    item_data["manufacturer"]      = row["manufacturer"]
    #m = re.search(r'[0-9\.]+"', row["name"])
    m = re.findall(r'[0-9\.]+"', row["name"])
    #item_data["screensize"]    = re.findall(r'[0-9\.]+"', row["name"])[0].replace("\"","")
    if len(m) >0:
        m = m[0].replace("\"","")
    else:
        m = 0
    #print(m)
    item_data["screensize"]    = m
    all_data2.append(item_data)
    #print(item_data)

results_pd2 = pd.DataFrame(all_data2)
results_pd2.to_csv(out_file, sep=',')
results_pd2.head()


,customerRating,department,departmentId,inStoreAvailability,itemId,largeImage,manufacturer,mediumImage,name,numReviews,onlineAvailability,quantityLimit,salePrice,screensize,thumbnailImage,upc
0,4.5,PHOTO/COMMODITIES,5,True,6337785,https://img.bbystatic.com/BestBuy_US/images/pr...,Solo,https://img.bbystatic.com/BestBuy_US/images/pr...,Solo - Classic Portfolio Laptop Case - Espresso,2.0,True,99.0,103.99,0,https://img.bbystatic.com/BestBuy_US/images/pr...,30918003312
1,4.7,PHOTO/COMMODITIES,5,True,7885027,https://img.bbystatic.com/BestBuy_US/images/pr...,SwissGear,https://img.bbystatic.com/BestBuy_US/images/pr...,SwissGear - Eva Laptop Tote - Black/Blue,167.0,True,3.0,89.99,0,https://img.bbystatic.com/BestBuy_US/images/pr...,7613329009888
2,4.6,PHOTO/COMMODITIES,5,True,9091126,https://img.bbystatic.com/BestBuy_US/images/pr...,Solo,https://img.bbystatic.com/BestBuy_US/images/pr...,Solo - Studio Portfolio Laptop Briefcase - Black,1772.0,True,10.0,35.99,0,https://img.bbystatic.com/BestBuy_US/images/pr...,30918006207
3,4.7,PHOTO/COMMODITIES,5,False,5369701,https://img.bbystatic.com/BestBuy_US/images/pr...,ToteIt!,https://img.bbystatic.com/BestBuy_US/images/pr...,ToteIt! - Deluxe Laptop Case - Purple,7.0,False,5.0,29.95,0,https://img.bbystatic.com/BestBuy_US/images/pr...,671196091273
4,4.3,PHOTO/COMMODITIES,5,False,5369711,https://img.bbystatic.com/BestBuy_US/images/pr...,ToteIt!,https://img.bbystatic.com/BestBuy_US/images/pr...,ToteIt! - Laptop Case - Black,3.0,False,5.0,14.99,0,https://img.bbystatic.com/BestBuy_US/images/pr...,671196088617


In [288]:
bestbuy_api_key = "Jz4AJitfYbGIGGaOuTR09Lpl"
bestbuyUrl = "https://api.bestbuy.com/v1/categories"

params2 = {
    "apiKey": bestbuy_api_key,
    "page": 1,
    #"show": ",".join(show_arr),
    "pageSize": 100,
    "format": "json" # json|xml
}
#params2["query"] = "ipad"
def bestbuy_categories():
    urlquery = bestbuyUrl
    bestbuy_data = requests.get(urlquery, params=params2)
    bestbuy_url = bestbuy_data.url
    bestbuy_data = bestbuy_data.json()
    #print_r(bestbuy_url)
    return bestbuy_data
print(bestbuyUrl)
ret = bestbuy_categories()
print_r(ret)


https://api.bestbuy.com/v1/categories
{
    "from": 1,
    "to": 100,
    "currentPage": 1,
    "total": 3861,
    "totalPages": 39,
    "queryTime": "0.031",
    "totalTime": "0.098",
    "partial": false,
    "canonicalUrl": "/v1/categories?pageSize=100&format=json&apiKey=Jz4AJitfYbGIGGaOuTR09Lpl",
    "categories": [
        {
            "id": "abcat0010000",
            "name": "Gift Ideas",
            "active": true,
            "url": "https://www.bestbuy.com/site/electronics/gift-ideas/abcat0010000.c?id=abcat0010000&cmp=RMX-cat",
            "path": [
                {
                    "id": "cat00000",
                    "name": "Best Buy"
                },
                {
                    "id": "abcat0010000",
                    "name": "Gift Ideas"
                }
            ],
            "subCategories": [
                {
                    "id": "pcmcat140000050035",
                    "name": "Capturing Photos & Videos"
                },
             

In [292]:
out_file = "bestbuy_categories_data_file.csv"
all_data=[]
ret = bestbuy_categories()
if "total" in ret:
    #print_r(ret)
    total = ret["total"]
    totalPages = ret["totalPages"]
    prices = "price:["+str(strt_price)+" TO "+str(end_price)+"]"
    all_data2 =[]
    for xx in range(1,(totalPages+1)):
    #for xx in range(1,(1+1)):
        page = xx
        all_data2 =[]
        params2["page"] = page
        #print(params2)
        time.sleep(1)
        if(page>1):
            ret = bestbuy_categories()
        print("Looking for categories page ("+str(page)+"), Total results ("+str(total)+") in ("+str(totalPages)+") Pages")
        for x in ret["categories"]:
            categories[x["id"]] = {
                "name":x["name"],
                "subCategories":{}
            }
            if "subCategories" in x:
                for i in x["subCategories"]:
                    categories[x["id"]]["subCategories"][i["id"]]=i["name"]

print("----------------------------------")
print("We Finished, but not yet")

results_pd2 = pd.DataFrame(categories)
#results_pd2.to_csv(out_file, sep=',')
results_pd2.head()


Looking for categories page (1), Total results (3861) in (39) Pages
Looking for categories page (2), Total results (3861) in (39) Pages
Looking for categories page (3), Total results (3861) in (39) Pages
Looking for categories page (4), Total results (3861) in (39) Pages
Looking for categories page (5), Total results (3861) in (39) Pages
Looking for categories page (6), Total results (3861) in (39) Pages
Looking for categories page (7), Total results (3861) in (39) Pages
Looking for categories page (8), Total results (3861) in (39) Pages
Looking for categories page (9), Total results (3861) in (39) Pages
Looking for categories page (10), Total results (3861) in (39) Pages
Looking for categories page (11), Total results (3861) in (39) Pages
Looking for categories page (12), Total results (3861) in (39) Pages
Looking for categories page (13), Total results (3861) in (39) Pages
Looking for categories page (14), Total results (3861) in (39) Pages
Looking for categories page (15), Total res

,abcat0010000,abcat0020004,abcat0100000,abcat0101000,abcat0101001,abcat0101005,abcat0102000,abcat0102003,abcat0102005,abcat0102008,...,pcmcat84000050000,pcmcat84000050001,pcmcat84000050004,pcmcat8400050004,pcmcat87800050001,pcmcat88400050003,pcmcat88800050003,pcmcat9400050001,pcmcat94300050028,sc_digital_platform
name,Gift Ideas,Unique Gifts,TV & Home Theater,TVs,All Flat-Screen TVs,TV/DVD Combos,Blu-ray & DVD Players,Blu-ray Players,DVD Players,Portable DVD Players,...,Weddings,Anniversaries,Baby Showers,Video Game Deals & Gaming Hardware on Sale,Customer Service,"Deals on MP3 Players, iPod & More",Deals on Car Audio & GPS Systems,Deals on TV Shows & Movies on DVD and Blu-ray,Birthdays,Digital Platform
subCategories,{'pcmcat140000050035': 'Capturing Photos & Vid...,{},"{'abcat0101000': 'TVs', 'abcat0103000': 'Smart...","{'abcat0101001': 'All Flat-Screen TVs', 'pcmca...",{'pcmcat138100050024': 'TV & Home Theater Serv...,{'abcat0107001': 'TV Accessories'},"{'abcat0102003': 'Blu-ray Players', 'pcmcat220...",{'pcmcat219300050004': 'Standard Blu-ray Playe...,{'pcmcat138100050024': 'TV & Home Theater Serv...,{},...,{},{},{},{},"{'pcmcat203400050001': 'Help', 'pcmcat29030005...",{},{},{},{},"{'dp-mobile-landing': 'DPMobileLanding', 'pcmc..."


In [294]:
print_r(categories)

{
    "abcat0010000": {
        "name": "Gift Ideas",
        "subCategories": {
            "pcmcat140000050035": "Capturing Photos & Videos",
            "pcmcat140000050036": "Listening to Digital Music",
            "pcmcat140000050037": "Computing Made Easy",
            "pcmcat140000050039": "Simple GPS Navigation",
            "pcmcat140000050040": "Playing Video Games",
            "pcmcat140000050041": "Watching HDTV",
            "pcmcat140000050042": "Enjoying Favorite Movies",
            "abcat0012000": "Him",
            "abcat0011000": "Her",
            "abcat0013000": "Teens",
            "abcat0014000": "Kids",
            "pcmcat84000050001": "Anniversaries",
            "pcmcat84000050004": "Baby Showers",
            "pcmcat94300050028": "Birthdays",
            "pcmcat84000050003": "Graduations",
            "pcmcat84000050000": "Weddings",
            "abcat0020004": "Unique Gifts",
            "pcmcat111900050004": "Pink and Red Gifts",
            "abcat0020006

In [290]:
categories = {}
for x in ret["categories"]:
    categories[x["id"]] = {
        "name":x["name"],
        "subCategories":{}
    }
    if "subCategories" in x:
        for i in x["subCategories"]:
            categories[x["id"]]["subCategories"][i["id"]]=i["name"]


In [291]:
print_r(categories)

{
    "abcat0010000": {
        "name": "Gift Ideas",
        "subCategories": {
            "pcmcat140000050035": "Capturing Photos & Videos",
            "pcmcat140000050036": "Listening to Digital Music",
            "pcmcat140000050037": "Computing Made Easy",
            "pcmcat140000050039": "Simple GPS Navigation",
            "pcmcat140000050040": "Playing Video Games",
            "pcmcat140000050041": "Watching HDTV",
            "pcmcat140000050042": "Enjoying Favorite Movies",
            "abcat0012000": "Him",
            "abcat0011000": "Her",
            "abcat0013000": "Teens",
            "abcat0014000": "Kids",
            "pcmcat84000050001": "Anniversaries",
            "pcmcat84000050004": "Baby Showers",
            "pcmcat94300050028": "Birthdays",
            "pcmcat84000050003": "Graduations",
            "pcmcat84000050000": "Weddings",
            "abcat0020004": "Unique Gifts",
            "pcmcat111900050004": "Pink and Red Gifts",
            "abcat0020006